In [1]:
import os
import json
import pandas as pd
import pysrt
import re
import shutil
from openai import OpenAI, RateLimitError
import yaml
import math

In [2]:
option = "paragraph_semantic_search"

In [3]:
def save_json(content, save_path):
    with open(save_path, 'w') as f:
        f.write(json.dumps(content))
def load_jsonl(filename):
    with open(filename, "r") as f:
        return [json.loads(l.strip("\n")) for l in f.readlines()]
def load_result_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    print(data)
    return data

In [4]:
json_folder = "/home/hlpark/REDUCE/REDUCE_benchmarks/HiREST/data/splits/tvqa"
train_audio_rel = load_jsonl(f'{json_folder}/train_audio_relevance_score_{option}.json')
with open(f'{json_folder}/all_data_train.json') as f:
    train_json = json.load(f)

In [5]:
upper_threshold = 3
lower_threshold = 1
train_audio_high, train_audio_low = {}, {}
for value in train_audio_rel[0]:
    if value['video'] in train_json[value['question']]:
        if 'score' not in value:
            continue
        if value['score'] > upper_threshold:
            train_audio_high[value['question']] = train_json[value['question']]
        elif value['score'] < lower_threshold:
            train_audio_low[value['question']] = train_json[value['question']]

In [6]:
print(len(train_audio_high))
print(len(train_audio_low))

39930
22530


In [7]:
import random
# randomly sample 200 from medical queries and 200 from non-medical queries

train_audio_high_len = len(train_audio_high)

random.seed(9)
train_audio_high_idx = random.sample(list(range(0, train_audio_high_len)), k = len(train_audio_low))
train_audio_high_sampled_queries = {}

# create json file
# if one video have different queries, change video name (append v1, v2, ...)

videos = []
cnt = 0
for idx in train_audio_high_idx:
    key  = list(train_audio_high.keys())[idx]
    train_audio_high_sampled_queries[key] = train_audio_high[key]

In [8]:
len(train_audio_high_sampled_queries)

22530

In [9]:
save_json(train_audio_low, f'{json_folder}/all_train_audio_low_{option}.json')
save_json(train_audio_high, f'{json_folder}/all_train_audio_high_{option}.json')
save_json(train_audio_high_sampled_queries, f'{json_folder}/sampled_train_audio_high_{option}.json')